**Метод Якоби**

Метод Якоби — итерационный алгоритм для вычисления собственных значений
и собственных векторов вещественной симметричной матрицы.

**Алгоритм**

Исходные данные: симметричная матрица $A$ (элементы $a_{ij}$).

Вычисляемые данные: диагональная матрица $\Lambda$ (элементы
$\lambda_{ij}$).

Матрица $A_{i+1}$ получается из $A_i$ по формуле
$A_{i+1}=J_i^{T} A_i J_i$, где $J_i$ — ортогональная матрица, называемая
вращением Якоби. При подходящем выборе $J_i$ матрица $A_m$ для больших
$m$ будет близка к диагональной матрице $\Lambda$.

Матрица $J_i$ выбирается так, чтобы сделать нулями пару внедиагональных
элементов матрицы $A_{i+1}$. 

Обозначим $s = \sin(\theta)$ и $c = \cos(\theta)$. Тогда матрица
$A_{i+1}$ состоит из следующих элементов:

$$\begin{aligned}
a_{jj}^{(i+1)} &= c^2\, a_{jj}^{(i)}  -  2 s c\, a_{jk}^{(i)}  +  s^2\, a_{kk}^{(i)} \\
a_{kk}^{(i+1)} &= s^2 \,a_{jj}^{(i)}  +  2 s c\, a_{jk}^{(i)}  +  c^2 \, a_{kk}^{(i)} \\
a_{jk}^{(i+1)} &= a_{kj}^{(i+1)} = (c^2 - s^2 ) \, a_{jk}^{(i)}  +  s c \, (a_{kk}^{(i)} - a_{jj}^{(i)} ) \\
a_{jm}^{(i+1)} &= a_{mj}^{(i+1)} = c \, a_{jm}^{(i)}  -  s \, a_{km}^{(i)} & m \ne j,k \\
a_{km}^{(i+1)} &= a_{mk}^{(i+1)} = s \, a_{jm}^{(i)}  + c \, a_{km}^{(i)} & m \ne j,k \\
a_{ml}^{(i+1)} &= a_{ml}^{(i)} &m,l \ne j,k\end{aligned}$$

Можно выбрать $\theta$ так, чтобы $a_{jk}^{(i+1)} = 0$ и
$a_{kj}^{(i+1)} = 0$. Отсюда получим равенство
$$\frac{a_{jj}^{(i)} - a_{kk}^{(i)}}{2 a_{jk}^{(i)}} = \frac{c^2 - s^2}{2sc} = \frac{\cos(2\theta)}{\sin(2\theta)} = \operatorname{ctg}(2\theta) \equiv \tau.$$

Если $a_{jj}^{(i)} = a_{kk}^{(i)}$, то $\theta = \frac{\pi}{4}$.

Положим $t = \frac{s}{c} = \operatorname{tg}(\theta)$ и заметим, что
$t^2 + 2 t \tau - 1 = 0$. Решая квадратное уравнение, находим
$$t = \frac{\operatorname{sign}(\tau)}{|\tau| + \sqrt{1+\tau^2}}, \quad c = \frac{1}{\sqrt{1+t^2}}, \quad s = tc.$$
Выбор параметров $j$ и $k$ производится путем построчного циклического
обхода внедиагональных элементов матрицы $A$.

In [8]:
import numpy as np

n = 4
A = 2 * np.eye(n) - np.eye(n, k=1) - np.eye(n, k=-1)

def Jacobi_Rotation(A,j,k):
    J = np.eye(n)
    if A[j,j] == A[k,k]:
        theta = np.pi / 4
        J[j, j], J[j, k], J[k, j], J[k, k] = np.cos(theta), -np.sin(theta), np.sin(theta), np.cos(theta)   
    else:       
        tau = (A[j,j] - A[k,k]) / (2 * A[j,k]) # ctg(2a)
        tg = np.sign(tau) / (abs(tau) + np.sqrt(1 + tau**2)) # tg(a)
        cos = 1 / np.sqrt(1 + tg**2) # cos(a)
        sin = cos * tg # sin(a)
        J[j, j], J[j, k], J[k, j], J[k, k] = cos, -sin, sin, cos
    S = np.dot(np.transpose(J), np.dot(A,J))
    return S

def Jacobi(A):
    while True:
        for j in range(0,n-1):
            for k in range(j+1,n):
                A = Jacobi_Rotation(A,j,k) # A_{i+1}
                if (np.linalg.norm(A - np.diag(np.diag(A))) < 10**-6):
                    return A
print("Вещественная симметричная матрица A, для которой мы хотим найти собственные числа и векторы:")                
print(A, "\n")  
print("Вычисленная диагональная матрица:")
print(Jacobi(A), "\n")
print("Собственные числа вычисленные с помощью метода Якоби:")
print(np.diag(Jacobi(A)), "\n")
print("Собственные числа вычисленные с помощью numpy:")
print(np.linalg.eigvals(A))

Вещественная симметричная матрица A, для которой мы хотим найти собственные числа и векторы:
[[ 2. -1.  0.  0.]
 [-1.  2. -1.  0.]
 [ 0. -1.  2. -1.]
 [ 0.  0. -1.  2.]] 

Вычисленная диагональная матрица:
[[ 3.81966011e-01  5.90604092e-08  5.02580446e-09 -2.04540255e-12]
 [ 5.90604092e-08  3.61803399e+00  4.64663860e-11 -5.05591173e-17]
 [ 5.02580450e-09  4.64665020e-11  2.61803399e+00  0.00000000e+00]
 [-2.04543464e-12 -1.76096303e-16  1.09360000e-16  1.38196601e+00]] 

Собственные числа вычисленные с помощью метода Якоби:
[0.38196601 3.61803399 2.61803399 1.38196601] 

Собственные числа вычисленные с помощью numpy:
[3.61803399 2.61803399 0.38196601 1.38196601]
